In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import csv
import pandas as pd
import numpy as np

In [ ]:
############################################################################
######################  第一部分  #########################################
####################   下载文件  ######################################

In [ ]:
"""
web of science 下载ISSN账号
Incite:602281532@qq.com
密码：rank2016!

"""

In [3]:
browser = webdriver.Chrome('C:/Users/rk/Desktop/chromedriver.exe') #锁定浏览器
browser.get("http://jcr.incites.thomsonreuters.com/JCRJournalHomeAction.action")#打开主页面

In [ ]:
##################################################################################
#打开页面后，直接手动点击至下面的程序
#########################################################################

In [8]:
inputs=browser.find_elements_by_xpath('//*[@id="categorySelectionView"]/div/label/input')#查找所有学科复选框
inputs2=browser.find_elements_by_xpath('//*[@id="categorySelectionView"]/div/label/span')#查找所有学科，为提取学科名

In [9]:
inputs2[0].text

'ACOUSTICS'

In [10]:
counter=0

In [11]:
for index,la in enumerate(inputs[counter:]):#程序中断后会从中断的地方开始循环，若想每循环一次，counter都改变一次，则需要将counter设置在for循环体外
    la.click()#点击学科分类
    browser.find_element_by_xpath('//*[@id="skip-to-content"]/div/div[2]/div[1]/div[9]/div[3]/a[2]').click()#点击summit
    
    browser.find_element_by_xpath('//*[@id="action_export"]/a').click()#点击下载
    browser.find_element_by_xpath('//*[@id="popup"]/div/ul/li[2]/a').click()#点击csv
    category=inputs2[index].text
    la.click()#再次点击上面的学科分类以取消选择
    fil=r'C:\Users\rk\Downloads'+'\\'+category+'.csv'#文件名及路径
    while True:
        try:
            os.rename(r'C:\Users\rk\Downloads\JournalHomeGrid.csv',fil)#对下载的文件修改文件名
        except:
            time.sleep(1)
        else:
            break
    counter+=1   

In [ ]:
#####################################################################################
##############################  第二部分 ##################################
######################   合并文件，添加学科分类   ##################

In [ ]:
Folder_Path =r'C:\Users\rk\Downloads\2015JCR'        #要拼接的文件夹及其完整路径
SaveFile_Path =  r'C:\Users\rk\Downloads\allresult'       #拼接后要保存的文件路径
#SaveFile_Name = r'all.csv'              #合并后要保存的文件名
 
    #修改当前工作目录
os.chdir(Folder_Path)
#将该文件夹下的所有文件名存入一个列表
file_list = os.listdir() 

In [ ]:
file_list

In [ ]:
#读入csv中的数据，（导入检索式）
a = []
csv_reader=csv.reader(open(Folder_Path +'\\'+ file_list[0]))#打开第一个文件
for row in csv_reader:
    a.append(row)#将读入的数据赋值给列表a

In [ ]:
############首先对Mapp.csv原文件数据去重，删除数据重复的项目######################
lis_a = []#创建空白列表，存储文件内容
csv_reader=csv.reader(open(r'C:\Users\rk\Desktop\Mapp.csv'))

In [ ]:
for element in csv_reader:
    lis_a.append(element)#逐行读入数据，写入列表

In [ ]:
####################运用此程序的前提是：先在原文件里做好分类汇总####################################
final_lis=[]
counter=0
while True:#为了是for循环中的起始值counter的改变，for进入循环后，起始值不再改变，
    for item_1 in lis_a[counter:]:
        diction=[]
        lis_c=[]
        initial_s=0
        lis_c.append(item_1[1])#将第一个期刊加入列表，在此处加入是防止仅出现一次而忽略
        for item_2 in lis_a[counter+1:]:
            if item_1[0]==item_2[0]:
                lis_c.append(item_2[1])#将相同的期刊加入列表
                initial_s+=1 #每找到一个相同的就把initial_s加一，     
        diction.append(item_1[0])
        diction.append(lis_c)
        final_lis.append(diction)
        break#为了使for循环中的起始值改变，循环一次后利用break强制退出for循环，利用while进入下一次循环
    counter=counter+initial_s+1
    if counter>=len(lis_a):#为了使while循环跳出,当大于列表长度的时候跳出循环
        break

In [ ]:
final_lis

In [ ]:
final_lis[1]

In [ ]:
final_lis[1][1]

In [ ]:
savefilename=final_lis[0]

In [ ]:
coun=counter=0
for final_li in final_lis:
    savefilename=final_li[0]
    flag=0
    lis_name=[]#创建空白列表
    lis_name.append(['Rank','Full Journal Title','ISSN','Total Cites','Journal Impact Factor','Eigenfactor Score','category'])
    #print(lis_name)
    #counter+=1
    for file_lis in file_list:#循环处理过的软科-WOS映射表
        category=file_lis[:file_lis.find('.')]#截取WOS学科分类名
        category=category.lower()#将WOS学科分类名全部小写
        ele=[]#创建空白列表
        for element in final_li[1]:#循环软科分类下的WOS学科分类，使其全部小写
            ele.append(element.lower())
        final_li[1]=ele #全部小写的重新赋值给final_li[1]  
        if category in final_li[1]:#判断WOS学科分类是否在软科-WOS映射表中
            #print(1)
            flag=1
            e=[]
            csv_reader=csv.reader(open(Folder_Path +'\\'+ file_lis))
            for row in csv_reader:
                e.append(row)#将读入的数据赋值给列表
            for item in e[2:-2]:
                item.append(savefilename)#将软科学科分类名字添加到最后一列
                lis_name.append(item)
    if flag==1:    
        result=pd.DataFrame(np.array(lis_name))#将pd.DataFrame(np.array(myarray))写入list
        result.to_csv("C:/Users/rk/Desktop/result"+"/"+ savefilename+".csv",index=False,encoding='ANSI')#按照软科学科分类导出文件
        
        

In [ ]:
lis_name

In [ ]:
np.array(lis_name)